<a href="https://colab.research.google.com/github/xiaoyufan/nbme/blob/main/baseline_deberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NBME Baseline DeBERTa

## Configurations

In [1]:
class Config:
  batch_size = 16
  device_name = 'gpu'
  epochs = 1
  input_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Preprocessed'
  learning_rate = 1e-4
  mode = 'dev'
  model = 'microsoft/deberta-base'
  output_dir = '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output'
  sequence_max_length = 466
  tpu_cores = 1

## Packages

In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 188 µs (started: 2022-04-29 05:15:05 +00:00)


In [3]:
!pip install transformers
!pip install tokenizers

!pip install --upgrade git+https://github.com/xiaoyufan/nbme.git@e8aadf2b3fe5faa8bf1b3e884a4d2f4d7fbd026b

  Cloning https://github.com/xiaoyufan/nbme.git (to revision e8aadf2b3fe5faa8bf1b3e884a4d2f4d7fbd026b) to /tmp/pip-req-build-_jrmymtc
  Running command git clone -q https://github.com/xiaoyufan/nbme.git /tmp/pip-req-build-_jrmymtc
  Running command git rev-parse -q --verify 'sha^e8aadf2b3fe5faa8bf1b3e884a4d2f4d7fbd026b'
  Running command git fetch -q https://github.com/xiaoyufan/nbme.git e8aadf2b3fe5faa8bf1b3e884a4d2f4d7fbd026b
  Running command git checkout -q e8aadf2b3fe5faa8bf1b3e884a4d2f4d7fbd026b
time: 11.2 s (started: 2022-04-29 05:15:05 +00:00)


In [22]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn

from ast import literal_eval
from nbme_utils.location import locations_to_spans, spans_to_locations
from nbme_utils.prediction import logits_to_spans
from nbme_utils.scoring import span_micro_f1
from transformers import AutoConfig, AutoModel, AutoTokenizer, PreTrainedTokenizer
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

time: 266 ms (started: 2022-04-29 05:20:00 +00:00)


## TPU

In [5]:
if Config.device_name == 'tpu':
  assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

time: 1.07 ms (started: 2022-04-29 05:15:24 +00:00)


In [6]:
!pip install cloud-tpu-client==0.10 torch==1.11.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

  Using cached https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl (152.9 MB)
time: 4.95 s (started: 2022-04-29 05:15:24 +00:00)


In [7]:
if Config.device_name == 'tpu':
  import torch_xla.core.xla_model as xm
  import torch_xla.distributed.parallel_loader as pl
  import torch_xla.distributed.xla_multiprocessing as xmp
  
  from torch.utils.data.distributed import DistributedSampler

time: 2.04 ms (started: 2022-04-29 05:15:29 +00:00)


## Tokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained(Config.model)
tokenizer.save_pretrained(f'{Config.output_dir}/tokenizer')

('/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/vocab.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/merges.txt',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/CS7150 Deep Learning Project/Dataset/Output/tokenizer/tokenizer.json')

time: 8.67 s (started: 2022-04-29 05:15:29 +00:00)


## Data Loading

### Defining Dataset

In [9]:
train = pd.read_csv(f'{Config.input_dir}/train.csv')
valid = pd.read_csv(f'{Config.input_dir}/validate.csv')
test = pd.read_csv(f'{Config.input_dir}/test.csv')
train.shape, valid.shape, test.shape

((11342, 8), (2958, 8), (5, 6))

time: 185 ms (started: 2022-04-29 05:15:37 +00:00)


In [10]:
if Config.mode == 'dev':
  train = train.sample(n=40, random_state=0).reset_index(drop=True)
  valid = valid.sample(n=10, random_state=0).reset_index(drop=True)
train.shape, valid.shape, test.shape

((40, 8), (10, 8), (5, 6))

time: 9.05 ms (started: 2022-04-29 05:15:38 +00:00)


In [11]:
train.head()

,id,case_num,pn_num,feature_num,annotation,location,pn_history,feature_text
0,00225_008,0,225,8,"['drinks coffee', 'drinks energy drinks']","['461 467;483 489', '461 467;506 519']",17 y/o previously healthy male here with heart...,Caffeine-use
1,81979_803,8,81979,803,[],[],67yo woman presents with 3 weeks of difficulty...,Auditory-hallucination-once
2,31800_311,3,31800,311,[],[],HPI: Mr. Hamilton is a 35 year old male who pr...,No-blood-in-stool
3,20158_216,2,20158,216,['44'],['22 24'],"Ms Dolores Montgmery, 44 yo F c/o Irregular me...",44-year
4,21591_201,2,21591,201,[],[],Dolores Montgomery is a 44 year old female wit...,Last-Pap-smear-I-year-ago


time: 13.4 ms (started: 2022-04-29 05:15:38 +00:00)


In [12]:
test.head()

,id,case_num,pn_num,feature_num,pn_history,feature_text
0,00016_000,0,16,0,HPI: 17yo M presents with palpitations. Patien...,Family-history-of-MI-OR-Family-history-of-myoc...
1,00016_001,0,16,1,HPI: 17yo M presents with palpitations. Patien...,Family-history-of-thyroid-disorder
2,00016_002,0,16,2,HPI: 17yo M presents with palpitations. Patien...,Chest-pressure
3,00016_003,0,16,3,HPI: 17yo M presents with palpitations. Patien...,Intermittent-symptoms
4,00016_004,0,16,4,HPI: 17yo M presents with palpitations. Patien...,Lightheaded


time: 8.37 ms (started: 2022-04-29 05:15:38 +00:00)


In [13]:
def generate_labels(encoded: PreTrainedTokenizer, sample: pd.DataFrame):
  labels = torch.zeros(len(encoded['input_ids']))

  location_spans = locations_to_spans(literal_eval(sample['location']))

  for idx, (seq_id, offsets)in enumerate(zip(encoded['sequence_ids'],
                                             encoded['offset_mapping'])):
    # None for special tokens added around or between sequences,
    # 0 for tokens corresponding to words in the first sequence,
    # 1 for tokens corresponding to words in the second sequence when a pair of sequences was jointly encoded.
    # Labels are generated from patient notes, which are encoded as the first sequence.
    if seq_id != 0:
      labels[idx] = -1
      continue

    subtoken_start, subtoken_end = offsets

    if any([subtoken_start >= location_start and subtoken_end <= location_end
            for location_start, location_end in location_spans]):
      labels[idx] = 1

  return labels

class NBMEDataset(Dataset):
  def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, config: Config,
               testing: bool = False):
    self.data = data
    self.tokenizer = tokenizer
    self.config = config
    self.testing = testing

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx: int):
    sample = self.data.iloc[idx]

    encoded = self.tokenizer(
      sample['pn_history'],
      sample['feature_text'],
      # TODO: Compute max length of sequences
      max_length=self.config.sequence_max_length,
      padding='max_length',
      return_offsets_mapping=True,
    )
    encoded['sequence_ids'] = np.array(encoded.sequence_ids()).astype('float16')

    x = {k: torch.tensor(v, dtype=torch.long) for k, v in encoded.items()}

    if self.testing:
      return x

    y_true = generate_labels(encoded, sample)
    return x, y_true

time: 32 ms (started: 2022-04-29 05:15:38 +00:00)


In [14]:
train_dataset = NBMEDataset(train, tokenizer, Config)
valid_dataset = NBMEDataset(valid, tokenizer, Config)
test_dataset = NBMEDataset(test, tokenizer, Config, testing=True)

time: 1.39 ms (started: 2022-04-29 05:15:38 +00:00)


## Model

In [15]:
class NBMEDebertaBaseline(nn.Module):
  def __init__(self):
    super().__init__()

    config = AutoConfig.from_pretrained(Config.model, output_hidden_states=True)
    self.model = AutoModel.from_pretrained(Config.model, config=config)

    self.fc = nn.Linear(config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    hidden_states = self.model(input_ids=input_ids, attention_mask=attention_mask)
    logits = self.fc(hidden_states[0])
    logits = torch.sigmoid(logits)
    logits = logits.squeeze(-1)
    return logits

time: 7.18 ms (started: 2022-04-29 05:15:38 +00:00)


In [16]:
model = NBMEDebertaBaseline()

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


time: 3.47 s (started: 2022-04-29 05:15:38 +00:00)


## Training

In [17]:
criterion = nn.BCEWithLogitsLoss(reduction = "none")

def loss_fn(y_pred, y_true):
  loss = criterion(y_pred, y_true)
  loss = torch.masked_select(loss, y_true >= 0).mean()
  loss = loss.mean()
  return loss

time: 2.6 ms (started: 2022-04-29 05:15:41 +00:00)


### Training on GPU / CPU

In [26]:
def train_fn(model, device, data_loader):
  optimizer = torch.optim.AdamW(model.parameters(), lr=Config.learning_rate)

  all_loss = []

  for x, y_true in tqdm(data_loader):
    optimizer.zero_grad()

    input_ids = x['input_ids'].to(device)
    attention_mask = x['attention_mask'].to(device)
    y_true = y_true.to(device)

    logits = model(input_ids, attention_mask)
    loss = loss_fn(logits, y_true)
    print(f'Loss: {loss}')
    all_loss.append(loss)

    loss.backward()
    
    if Config.device_name == 'tpu':
      xm.optimizer_step(optimizer)
    else:
      optimizer.step()

def valid_fn(model, device, data_loader):
  model.eval()

  true_spans = []
  all_logits = []
  all_offsets = []
  all_sequence_ids = []
  
  for x, y_true in tqdm(data_loader):
    true_spans.append(y_true)
    
    input_ids = x['input_ids'].to(device)
    attention_mask = x['attention_mask'].to(device)

    logits = model(input_ids, attention_mask)

    all_logits.append(logits.detach().cpu().numpy())
    all_offsets.append(x['offset_mapping'].numpy())
    all_sequence_ids.append(x['sequence_ids'].numpy())
  
  all_logits = np.concatenate(all_logits, axis=0)
  all_offsets = np.concatenate(all_offsets, axis=0)
  all_sequence_ids = np.concatenate(all_sequence_ids, axis=0)
  true_spans = np.concatenate(true_spans, axis=0)

  pred_spans = logits_to_spans(all_logits, all_offsets, all_sequence_ids)
  true_spans = logits_to_spans(true_spans, all_offsets, all_sequence_ids)
  print(all_logits.shape)
  print(np.shape(pred_spans))
  print(np.shape(true_spans))
  
  score = span_micro_f1(pred_spans, true_spans)
  return score

time: 40.8 ms (started: 2022-04-29 05:27:16 +00:00)


### Training on TPU

In [27]:
def run_tpu(index):
  device = xm.xla_device()
  print(f'Process {index} is using {xm.xla_real_devices([str(device)])[0]}')

  model.to(device)

  train_sampler = DistributedSampler(
    train_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=True)
  valid_sampler = DistributedSampler(
    valid_dataset,
    num_replicas=xm.xrt_world_size(),
    rank=xm.get_ordinal(),
    shuffle=False)

  train_loader = DataLoader(
    train_dataset,
    batch_size=Config.batch_size,
    sampler=train_sampler,
    drop_last=True)
  valid_loader = DataLoader(
    valid_dataset,
    batch_size=Config.batch_size,
    sampler=valid_sampler,
    drop_last=True)

  train_loader = pl.MpDeviceLoader(train_loader, device)
  valid_loader = pl.MpDeviceLoader(valid_loader, device)

  for epoch in range(Config.epochs):
    train_fn(model, device, train_loader)
    
    with torch.no_grad():
      valid_fn(model, device, valid_loader)

  # Barrier to prevent master from exiting before workers connect.
  xm.rendezvous('init')

if Config.device_name == 'tpu':
  xmp.spawn(run_tpu, args=(), nprocs=Config.tpu_cores, start_method='fork')

time: 15.1 ms (started: 2022-04-29 05:27:16 +00:00)


## Testing

In [28]:
def test_fn(model, device, data_loader):
  model.eval()

  all_logits = []
  all_offsets = []
  all_sequence_ids = []
  
  for x in tqdm(data_loader):
    input_ids = x['input_ids'].to(device)
    attention_mask = x['attention_mask'].to(device)

    with torch.no_grad():
      logits = model(input_ids, attention_mask)

    all_logits.append(logits.detach().cpu().numpy())
    all_offsets.append(x['offset_mapping'].numpy())
    all_sequence_ids.append(x['sequence_ids'].numpy())
  
  all_logits = np.concatenate(all_logits, axis=0)
  all_offsets = np.concatenate(all_offsets, axis=0)
  all_sequence_ids = np.concatenate(all_sequence_ids, axis=0)

  all_spans = logits_to_spans(all_logits, all_offsets, all_sequence_ids)
  locations = [spans_to_locations(spans) for spans in all_spans]
  return locations

time: 9.75 ms (started: 2022-04-29 05:27:16 +00:00)


In [29]:
def run_non_tpu():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print(f'Using {device}')

  model.to(device)

  train_loader = DataLoader(
      train_dataset,
      batch_size=Config.batch_size,
      shuffle=True)
  valid_loader = DataLoader(
      valid_dataset,
      batch_size=Config.batch_size,
      shuffle=False)
  test_loader = DataLoader(
      test_dataset,
      batch_size=Config.batch_size,
      shuffle=False)

  for epoch in range(Config.epochs):
    print(f'Epoch: {epoch}')
    
    train_fn(model, device, train_loader)
    valid_fn(model, device, valid_loader)
  
  test_fn(model, device, test_loader)

if Config.device_name != 'tpu':
  run_non_tpu()

Using cpu


  0%|          | 0/3 [00:00<?, ?it/s]

Loss: 0.69315105676651
Loss: 0.693148672580719
Loss: 0.693148136138916


  0%|          | 0/1 [00:00<?, ?it/s]

(10, 466)
(10, 0)
(10,)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


  0%|          | 0/1 [00:00<?, ?it/s]

time: 1min 26s (started: 2022-04-29 05:27:16 +00:00)
